In [ ]:
@dlt.table(
    comment="load data to sales orders fact table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def fact_sales_orders():
    s = dlt.read("sales_orders_cleansed").alias("s")
    p = dlt.read("dim_products").alias("p")
    c = dlt.read("dim_customers").alias("c")
    return (
        s.join(p, s.product_id == p.product_id, "inner")
        .join(c, s.customer_id == c.customer_id, "inner")
        .select(
            "s.order_number",
            "c.customer_key",
            "p.product_key",
            col("s.order_datetime").cast("date").alias("order_date"),
            "s.unit_price",
            "s.quantity",
            expr("s.unit_price * s.quantity").alias("total_price"),
        )
    )

@dlt.table(
    comment="load data to customer sales fact table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def fact_customer_sales():
    s = dlt.read("sales_orders_cleansed").alias("s")
    p = dlt.read("dim_products").alias("p")
    c = dlt.read("dim_customers").alias("c")
    return (
        s.join(p, s.product_id == p.product_id, "inner")
        .join(c, s.customer_id == c.customer_id, "inner")
        .groupBy("c.customer_key", "p.product_key")
        .agg(
            sum("quantity").alias("total_quantity"),
            sum(expr("s.unit_price * s.quantity")).alias("sale_amount"),
        )
    )
     